In [1]:
import os,sys
os.chdir("../")
path = os.getcwd()
sys.path.insert(0, path)
import scipy as sp
import scipy.integrate as spint8
import GWGen; from GWGen import WFGenerator
from GWGen.WFGenerator import *
from few.utils.utility import get_separatrix
from superrad import ultralight_boson;
import time
import cProfile, pstats
from pstats import SortKey

M = 1e6
mu = 1e1
procamass=1e-18 # eV
a = 0.6
p0 = 10.0
e0 = 0.2
iota0 = 0.0
Y0 = np.cos(iota0)
Phi_phi0 = 0.
Phi_theta0 =0.
Phi_r0 = 0.
qS = 0.2
phiS = 0.2
qK = 0.8
phiK = 0.8
dist = 1.0
mich = False
dt = 15.0
T = 2

In [2]:
diffeq = PN(M,mu)
pr = cProfile.Profile()
pr.enable()
diffeq(0,[6.,0.7,0,0])
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

         151 function calls (147 primitive calls) in 0.020 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.016    0.016    0.016    0.016 {pyUtility.pyGetSeparatrix}
        1    0.003    0.003    0.020    0.020 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/few/utils/utility.py:363(get_separatrix)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        1    0.000    0.000    0.000    0.000 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/Utils/Kerr.py:82(<lambda>)
        1    0.000    0.000    0.020    0.020 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/WFGenerator/Kludge.py:102(__call__)
        1    0.000    0.000    0.000    0.000 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/Utils/Kerr.py:87(<lambda>)
        1    0.000    0.000    0.000    0.000 {pyKerrFreqs.pyKerrGeoCoordinateFrequencies}
        1    0.000    0.000

In [4]:
traj = PNTraj()
pr = cProfile.Profile()
pr.enable()
traj(M,mu,a,p0,e0,Y0, npoints=120,dense_output=True)
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

         238745 function calls (229951 primitive calls) in 0.685 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2036    0.222    0.000    0.240    0.000 {pyUtility.pyGetSeparatrix}
     1907    0.080    0.000    0.085    0.000 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/Utils/Kerr.py:82(<lambda>)
     1907    0.065    0.000    0.068    0.000 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/Utils/Kerr.py:87(<lambda>)
     1907    0.043    0.000    0.045    0.000 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/Utils/Kerr.py:72(<lambda>)
     1907    0.036    0.000    0.038    0.000 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/Utils/Kerr.py:77(<lambda>)
     1907    0.027    0.000    0.570    0.000 /mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/WFGenerator/Kludge.py:102(__call__)
19742/10951    0.024    0.000    0.071    0.000 {built-in method numpy.core._multiarray_umath.implement_array_functi

In [4]:
use_gpu = False
inspkw = {"max_init_len": int(1e3),"npoints":100}
sumkwargs = {"use_gpu": use_gpu, "pad_output": False}
emriwop = EMRIWaveform(inspiral_kwargs=inspkw.copy(),sum_kwargs=sumkwargs.copy(),use_gpu=use_gpu)
pr = cProfile.Profile()
pr.enable()
emriwop(M,mu,a,p0,e0,Y0,qS,phiS,qK,phiK,dist,T=T)
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

/mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/few/summation/aakwave.py:228: UserWarning: Inclination trajectory includes values within 1e-6 of the poles. We shift these values automatically away from poles by 1e-6.
  "Inclination trajectory includes values within 1e-6 of the poles. We shift these values automatically away from poles by 1e-6."


         193898 function calls (186796 primitive calls) in 2.610 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.820    1.820    1.820    1.820 {pycpuAAK.pyWaveform}
     1716    0.305    0.000    0.320    0.000 {pyUtility.pyGetSeparatrix}
     1607    0.069    0.000    0.073    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:82(<lambda>)
        2    0.061    0.031    0.061    0.031 {method 'searchsorted' of 'numpy.ndarray' objects}
     1607    0.048    0.000    0.050    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:87(<lambda>)
     1607    0.038    0.000    0.040    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/GWGen/Utils/Kerr.py:72(<lambda>)
     1607    0.032    0.000    0.033    0.000 /mnt/Data_Volume/Documents/software/Anaconda

In [3]:
ub = ultralight_boson.UltralightBoson(spin=1,model="relativistic");
emrip = EMRIWithProcaWaveform(inspiral_kwargs=inspkw.copy(), sum_kwargs=sumkwargs.copy(), use_gpu=False)
traj = PNTraj()

In [4]:
pr = cProfile.Profile()
pr.enable()
emrip(M,mu,procamass,a,p0,e0,Y0,T=T,UltralightBoson=ub,qS=qS,qK=qK,phiS=phiS,phiK=phiK,dist=dist,npoints=100)
pr.disable()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats('tottime')
ps.print_stats()

****************
Generating Trajectory for final black mass and spin 995730.0420975242 and 0.02980664747609163
****************
Integration halted before ending time. Reasons: Separatrix reached!
final mass 995730.0420975242
final spin 0.02980664747609163
****************
Generating Waveform
****************
         326195 function calls (319811 primitive calls) in 4.421 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     5564    2.173    0.000    2.173    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/scipy/interpolate/fitpack2.py:1054(integral)
        1    1.225    1.225    1.225    1.225 {pycpuAAK.pyWaveform}
     1512    0.111    0.000    0.121    0.000 {pyUtility.pyGetSeparatrix}
     4173    0.075    0.000    0.075    0.000 /mnt/Data_Volume/Documents/software/Anaconda/envs/few_env/lib/python3.7/site-packages/superrad/matched_waveform.py:104(mass_cloud)
     5657    0.072